In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm import tqdm
import xgboost as xgb
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy import sparse
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
import re

In [35]:
#NUM_TRAIN = 2 # 139256
#
#data_dir = Path('../raw_data/AI4Code')
#
#def read_notebook(path):
#    return (
#        pd.read_json(
#            path,
#            dtype={'cell_type': 'category', 'source': 'str'})
#        .assign(id=path.stem)
#        .rename_axis('cell_id')
#    )
#
#paths_train = list((data_dir / 'train_data').glob('*.json'))[:NUM_TRAIN]
#notebooks_train = [read_notebook(path) for path in tqdm(paths_train, desc='Train NBs')]
#df = pd.concat(notebooks_train).set_index('id', append=True).swaplevel().reset_index()
#df

In [36]:
df = pd.read_csv(f'../data/10k_preproc.csv')
#df = df.drop()
df.head()

,Unnamed: 0,cell_id,id,cell_type,source,rank,language,cell_order_training,about,above,...,x_train,xlabel,y_pred,y_test,y_train,year,ylabel,your,ancestor_id,parent_id
0,0,8f84d7a9,000757b90aaca0,0,import panda import spacy import networkx real...,3,en,3,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ff3e6f37,NaN
1,1,eb6ca769,000757b90aaca0,0,spacy load en_core_web_lg more detailed model ...,5,en,5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ff3e6f37,NaN
2,2,bc595bc2,000757b90aaca0,0,rcparams figure figsize make output plot large...,6,en,6,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ff3e6f37,NaN
3,3,93cceeef,000757b90aaca0,0,rowlimit this limit tweet manageable number da...,8,en,8,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ff3e6f37,NaN
4,4,3cb3d383,000757b90aaca0,0,data head,9,en,9,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ff3e6f37,NaN


In [37]:
df['rank'].isna().sum()

0

In [38]:
df = df.reset_index(drop=True)

In [39]:
df.head()

,Unnamed: 0,cell_id,id,cell_type,source,rank,language,cell_order_training,about,above,...,x_train,xlabel,y_pred,y_test,y_train,year,ylabel,your,ancestor_id,parent_id
0,0,8f84d7a9,000757b90aaca0,0,import panda import spacy import networkx real...,3,en,3,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ff3e6f37,NaN
1,1,eb6ca769,000757b90aaca0,0,spacy load en_core_web_lg more detailed model ...,5,en,5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ff3e6f37,NaN
2,2,bc595bc2,000757b90aaca0,0,rcparams figure figsize make output plot large...,6,en,6,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ff3e6f37,NaN
3,3,93cceeef,000757b90aaca0,0,rowlimit this limit tweet manageable number da...,8,en,8,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ff3e6f37,NaN
4,4,3cb3d383,000757b90aaca0,0,data head,9,en,9,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ff3e6f37,NaN


In [40]:
df.drop(columns='Unnamed: 0', inplace=True)

In [41]:
def preprocess_text(text):
    # Remove all the special characters
    text = re.sub(r'\W', ' ', text)
    # remove all single characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    # Remove single characters from the start
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text)
    # Substituting multiple spaces with single space
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    # Removing prefixed 'b'
    text = re.sub(r'^b\s+', '', text)
    # Converting to Lowercase
    text = text.strip().lower()

    #return text
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    tokens = [word for word in tokens if len(word) > 3]
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer

def preprocess_df(df, min_df=0.01):
    df['cell_type'] = df['cell_type'].map({'code': 0, 'markdown': 1})

    tqdm.pandas(desc="preprocessing du texte")
    df['source'] = df['source'].progress_apply(preprocess_text)

    vectorizer = TfidfVectorizer(min_df=min_df)
    source_vect = vectorizer.fit_transform(tqdm(df['source'], desc="Vectorisation"))
    source_vect = pd.DataFrame(
        source_vect.toarray(),
        columns = vectorizer.get_feature_names_out()
    )

    df = pd.concat([df, source_vect], axis=1)

    return df

In [43]:
#df_preproc = preprocess_df(df)
#df_preproc['id'].nunique()

#ancestors = pd.read_csv('../raw_data/AI4Code/train_ancestors.csv')
#ancestors['id'].nunique()

#df_preproc_2 = pd.merge(df_preproc, ancestors, how='left', on='id')
#df_preproc_2.head()

In [44]:
#df_preproc_2.to_csv('../data/10k_preproc')

In [45]:
from sklearn.model_selection import GroupShuffleSplit

def train_val_split(df, test_size=0.1):
    df = df.reset_index().set_index('id')
    splitter = GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=0)

    # Split, keeping notebooks with a common origin (ancestor_id) together
    ids = df.index.unique('id')
    print(ids, ids.shape)
    ancestors = df.loc[ids, 'ancestor_id'].reset_index().drop_duplicates().set_index('id')
    print(ancestors, ancestors.shape)
    ids_train, ids_valid = next(splitter.split(ids, groups=ancestors))
    ids_train, ids_valid = ids[ids_train], ids[ids_valid]

    df_train = df.loc[ids_train, :]
    df_valid = df.loc[ids_valid, :]

    return df_train, df_valid

In [46]:
df_train, df_val = train_val_split(df)

Index(['000757b90aaca0', '000af347c31e72', '000fce1d6fbb6a', '002b5d330ee1ec',
       '002f895d088a9e', '004f5132c0b01c', '005f1abed1ba8a', '0064d9181701a2',
       '0074d80f6910fa', '0077214cf95e83',
       ...
       'ffc7196c36d684', 'ffccaad7a9a626', 'ffd5a56a09bb95', 'ffd60f1ce95983',
       'ffda671486b0d6', 'ffe20f3620057d', 'ffe69d93f7d21d', 'ffe8d0aa5e7d68',
       'fff253383c48e3', 'fffbd2e276f908'],
      dtype='object', name='id', length=9000) (9000,)
               ancestor_id
id                        
000757b90aaca0    ff3e6f37
000af347c31e72    e94552a4
000fce1d6fbb6a    b2f39261
002b5d330ee1ec    7cc203ad
002f895d088a9e    de1c123a
...                    ...
ffe20f3620057d    9ab0faf5
ffe69d93f7d21d    4df0d0c8
ffe8d0aa5e7d68    7d647f1d
fff253383c48e3    07590ca9
fffbd2e276f908    5b77f02e

[9000 rows x 1 columns] (9000, 1)


In [47]:
df_train.drop(columns=['cell_id', 'level_0', 'source', 'language', 'ancestor_id', 'parent_id'], inplace=True)

In [48]:
df_val.drop(columns=['cell_id', 'level_0', 'source', 'language', 'ancestor_id', 'parent_id'], inplace=True)
df_val.reset_index(inplace=True)
df_val.head()

,id,cell_type,rank,cell_order_training,about,above,accuracy,accuracy_score,after,also,...,word,x_test,x_train,xlabel,y_pred,y_test,y_train,year,ylabel,your
0,000757b90aaca0,0,3,3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,000757b90aaca0,0,5,5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.291537,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,000757b90aaca0,0,6,6,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,000757b90aaca0,0,8,8,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,000757b90aaca0,0,9,9,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Preprocessing - Normalisation du rang

In [49]:
df_train['rank'] = df_train.groupby('id')['rank'].transform(lambda x: 0 if x.max() == 0 else x / x.max())
df_val['rank'] = df_val.groupby('id')['rank'].transform(lambda x: 0 if x.max() == 0 else x / x.max())

In [50]:
df_train['cell_order_training'] = np.where(df_train['cell_type'] == 'markdown', 0, df_train['rank'])
df_val['cell_order_training'] = np.where(df_val['cell_type'] == 'markdown', 0, df_val['rank'])

In [51]:
df.loc[df['id'] == 'fffbde276f908']

,cell_id,id,cell_type,source,rank,language,cell_order_training,about,above,accuracy,...,x_train,xlabel,y_pred,y_test,y_train,year,ylabel,your,ancestor_id,parent_id


# TRAINING

In [52]:
def train_XGBRanker():

    config_defaults = {"booster":'gbtree',
                   "objective":'rank:pairwise',
                   "random_state":24,
                   "learning_rate":0.1,
                   "n_estimators":110}

    # Initiate the model
    model = XGBRanker(booster=config.booster,
                      objective=config.objective,
                      random_state=config.random_state,
                      learning_rate=config.learning_rate,
                      n_estimators=config.n_estimators)

    # Train the model
    model.fit(X_train, y_train, group=groups, verbose=True)

    # Create df containing the cell_id and the prediction
    predict = pd.DataFrame({"cell_id" : df_valid["cell_id"],
                            "pred" : model.predict(X_valid)}, index = df_valid.index)

    # Sort (using the predicted rank) and then group
    predict = predict.sort_values(by = ['id', 'pred'], ascending = [False, True])\
                        .groupby('id')['cell_id'].apply(list)

    # Create the same but for actual data
    actual = df_valid.sort_values(by = ['id', 'rank'], ascending = [False, True])\
                            .groupby('id')['cell_id'].apply(list)

    # Kendall Metric
    metric = kendall_tau(actual, predict)
    print(clr.S+"Kendall Tau"+clr.E, metric)
    wandb.log({"kendall_tau": np.float(metric)})

In [53]:
# Groupes pour l'entraînement
groups_train = df_train.groupby('id').size().to_numpy()
groups_val = df_val.groupby('id').size().to_numpy()

In [54]:
# Préparation des étiquettes d'entraînement
y_train = df_train['rank'].to_numpy()
y_val = df_val['rank'].to_numpy()

# Préparation des caractéristiques finales
df_train.reset_index(drop=True, inplace=True)
df_val.reset_index(drop=True, inplace=True)

X_train = df_train.drop(columns=['rank'])
X_val = df_val.drop(columns=['rank'])

In [55]:
y_train

array([0.        , 0.18181818, 0.27272727, ..., 0.        , 0.        ,
       0.        ])

In [56]:
X_train

,cell_type,cell_order_training,about,above,accuracy,accuracy_score,after,also,analysis,append,...,word,x_test,x_train,xlabel,y_pred,y_test,y_train,year,ylabel,your
0,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.091891,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0,0.181818,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0,0.272727,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.317652,0.301438,0.0,0.0,0.0,0.306852,0.0,0.0,0.0
3,0,0.363636,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.352998,0.0,0.0,0.0,0.359338,0.0,0.0,0.0
4,0,0.454545,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.712323,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374068,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
374069,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.252021,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
374070,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
374071,1,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [57]:
# Initialiser le modèle
model = xgb.XGBRanker(
    min_child_weight=10,
    subsample=0.5,
    tree_method='hist',
)

# Entraîner le modèle
model.fit(X_train, y_train, group=groups_train)


XGBoostError: [21:15:38] /workspace/src/common/ranking_utils.h:348: Check failed: label_is_integer: When using relevance degree as target, label must be either 0 or positive integer.
Stack trace:
  [bt] (0) /home/lionel/.pyenv/versions/3.10.6/envs/google_ai4_code/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x1ba24e) [0x7ff37806124e]
  [bt] (1) /home/lionel/.pyenv/versions/3.10.6/envs/google_ai4_code/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x261474) [0x7ff378108474]
  [bt] (2) /home/lionel/.pyenv/versions/3.10.6/envs/google_ai4_code/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x60587b) [0x7ff3784ac87b]
  [bt] (3) /home/lionel/.pyenv/versions/3.10.6/envs/google_ai4_code/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x55f9ed) [0x7ff3784069ed]
  [bt] (4) /home/lionel/.pyenv/versions/3.10.6/envs/google_ai4_code/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x4900d1) [0x7ff3783370d1]
  [bt] (5) /home/lionel/.pyenv/versions/3.10.6/en

In [ ]:

# Exemple de prédiction
y_pred = model.predict(X_test)

print(y_pred)